In [74]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout, MaxPooling1D, GRU, Flatten
from sklearn.preprocessing import MinMaxScaler
from keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping
import tensorflow
import numpy as np
from keras.layers import Input, Embedding, Conv1D, LSTM, Dense
from keras.models import Model

In [75]:
#Read data from csv
df = pd.read_csv("cosas_finales\casos_all - obtiene80acierto.txt", error_bad_lines=False)
#df = df.apply(pd.to_numeric)
df

C:\Users\migue\AppData\Local\Temp\ipykernel_6728\582951787.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("cosas_finales\casos_all - obtiene80acierto.txt", error_bad_lines=False)


,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,35714,0.020,178573,0.014,357147,5.000000e-03,0
1,28007,0.006,140036,0.002,280072,0.000000e+00,0
2,36722,0.017,183612,0.013,367225,3.000000e-03,0
3,27054,0.028,135272,0.013,270545,2.200000e-02,0
4,23816,0.012,119082,0.011,238164,2.800000e-02,0
...,...,...,...,...,...,...,...
419995,49,7058.940,248,1525299.200,496,2.440479e+07,5
419996,46,4866.800,232,3871022.080,464,1.398563e+07,5
419997,59,36968.220,296,3112120.320,592,4.149494e+06,5
419998,79,152842.090,397,3754246.380,795,1.021484e+07,5


In [76]:
#clean wrong values

# for i in range(0, len(df)-1):
#   #print(str(df.tipo.values[i]/) + " " + str(i))
#   if df.tipo[i] < 0 or df.tipo[i] > 5:
#     df = df.drop(i, axis=0)

# df = df.dropna()
# df

In [77]:
#drop type column to obtain the x_train data use 6 if size included 3 if not
x_train = df.iloc[:, 0:6]
#x_train = df.iloc[:, 0:3]

#y_train = df.iloc[:,6]
x_train

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2
0,35714,0.020,178573,0.014,357147,5.000000e-03
1,28007,0.006,140036,0.002,280072,0.000000e+00
2,36722,0.017,183612,0.013,367225,3.000000e-03
3,27054,0.028,135272,0.013,270545,2.200000e-02
4,23816,0.012,119082,0.011,238164,2.800000e-02
...,...,...,...,...,...,...
419995,49,7058.940,248,1525299.200,496,2.440479e+07
419996,46,4866.800,232,3871022.080,464,1.398563e+07
419997,59,36968.220,296,3112120.320,592,4.149494e+06
419998,79,152842.090,397,3754246.380,795,1.021484e+07


In [78]:
y_train = []
for i in range(0,len(df)):
  high = df.tipo[i]
  if high == 0:
    y_train.append([1,0,0,0,0,0])
  elif high == 1:
    y_train.append([0,1,0,0,0,0])
  elif high == 2:
    y_train.append([0,0,1,0,0,0])
  elif high == 3:
    y_train.append([0,0,0,1,0,0])
  elif high == 4:
    y_train.append([0,0,0,0,1,0])
  elif high == 5:
    y_train.append([0,0,0,0,0,1])
  else:
#    x_train.delete(i)
    print(str(i) + " " + str(df.tipo[i]))

len(y_train)

420000

In [79]:
#reshape data to the type the AI model needs
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

In [80]:
real_df = pd.read_csv("cosas_finales/tiempos_test.csv", error_bad_lines=False)
x_train_val = real_df.iloc[:, 1:7]

y_train_val = []
for i in range(0,len(real_df)):
  high = real_df.tipo[i]
  if high == 0:
    y_train_val.append([1,0,0,0,0,0])
  elif high == 1:
    y_train_val.append([0,1,0,0,0,0])
  elif high == 2:
    y_train_val.append([0,0,1,0,0,0])
  elif high == 3:
    y_train_val.append([0,0,0,1,0,0])
  elif high == 4:
    y_train_val.append([0,0,0,0,1,0])
  elif high == 5:
    y_train_val.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipo[i]))

x_train_val = np.array(x_train_val)
y_train_val = np.array(y_train_val)

C:\Users\migue\AppData\Local\Temp\ipykernel_6728\373423020.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  real_df = pd.read_csv("cosas_finales/tiempos_test.csv", error_bad_lines=False)


Note, the most confusing thing here is that the shape of the input to the model is defined as an argument on the first hidden layer. This means that the line of code that adds the first layer is doing 2 things, defining the input or visible layer and the first hidden layer.

In [94]:
#Generate a sequential model with 4 layers 
#The last one has 6 nodes, one for each execution time order [cte, logn, n, nlogn, n2, n3]
#The ones in between have arbitrary values obtain through testing ( still may be swapped )

#redes GRU igual esta interesante

model = Sequential()
#model.add(no. de entradas de cada caso) 
#esto es asi, en la libreria keras, en el modelo secuencial, se define en input_shape el nº de nodos de la entrada
model.add(Dense(32, activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))

#model.add(Conv1D(64, 3, activation='relu'))
#model.add(MaxPooling1D(2))
#model.add(LSTM(200, return_sequences=True))
#model.add(LSTM(200, return_sequences=True))
#model.add(LSTM(200, return_sequences=True))
#model.add(LSTM(200))

#model.add(Dense(50))
model.add(Dense(6, activation = 'softmax'))

#model.add(Dense(50, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dense(25, activation='relu'))
#model.add(Dense(5, activation='relu'))

#Compile the model previously created with the chosen parameters
#optimizer='sgd'
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 6, 32)             64        
                                                                 
 lstm_11 (LSTM)              (None, 6, 32)             8320      
                                                                 
 lstm_12 (LSTM)              (None, 32)                8320      
                                                                 
 dense_14 (Dense)            (None, 6)                 198       
                                                                 
Total params: 16,902
Trainable params: 16,902
Non-trainable params: 0
_________________________________________________________________


In [95]:
#Train the model with the 'train' portion of the data 
#The other parameters are tuned down in order to reduce train time but should be higher
callback = EarlyStopping(monitor='val_accuracy', patience = 3, restore_best_weights=True)
model.fit(x_train, y_train, batch_size=1, epochs=1, validation_data=(x_train_val, y_train_val))

#model.fit(x_train, y_train, batch_size=16, epochs=1, validation_data=(x_train_val, y_train_val))


420000/420000 [==============================] - 1436s 3ms/step - loss: 0.9332 - accuracy: 0.4991 - val_loss: 7.9115 - val_accuracy: 0.0000e+00


In [ ]:
#Save trained model to a file
model.save("modelRNCN.h5")

In [ ]:
#Use the model to predict the test data in order to find the success percent in the test data

predictions = model.predict(x_train_val)
predictions

array([[0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.19331636, 0.1042387 , 0.16334958, 0.13785027, 0.23995358,
        0.16129154],
       [0.1941095 , 0

In [ ]:
real_preds = []
for i in range(0,len(predictions)):
  p = predictions[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0,0,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1,0,0])
  elif max(p) == p[4]:
    real_preds.append([0,0,0,0,1,0])
  elif max(p) == p[5]:
    real_preds.append([0,0,0,0,0,1])

#Calculate error generated in the "real"predictions of test data
err = real_preds - y_train_val
err

array([[-1,  0,  0,  0,  1,  0],
       [ 0, -1,  0,  0,  1,  0],
       [ 0, -1,  0,  0,  1,  0],
       [ 0, -1,  0,  0,  1,  0],
       [ 0, -1,  0,  0,  1,  0],
       [ 0,  0, -1,  0,  1,  0],
       [ 0,  0, -1,  0,  1,  0],
       [ 0,  0, -1,  0,  1,  0],
       [ 0,  0, -1,  0,  1,  0],
       [ 0,  0,  0, -1,  1,  0],
       [ 0,  0,  0, -1,  1,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0]])

In [ ]:
def numToType( t):
  if t==0: return "CTE"
  elif t==1:return "LOG"
  elif t==2:return "N"
  elif t==3:return "NLOGN"
  elif t==4:return "N^2"
  elif t==5:return "N^3"

error_count = 0
for i in range(0, len(err)):
  if max(err[i]) != min(err[i]):
    print(str(real_df.name.values[i]) + " sol: " + numToType(real_df.tipo.values[i]) + "  pred: " + numToType(np.argmax(real_preds[i])))
    error_count += 1
print(str(error_count) + " / " + str(len(real_preds)) + " => real error% of " + str(error_count/len(real_preds)*100))

-114 sol: CTE  pred: N^2
-cajasBolas sol: LOG  pred: N^2
-167fractales sol: LOG  pred: N^2
--OJ10019 sol: LOG  pred: N^2
--323 sol: LOG  pred: N^2
niniosPrimero sol: N  pred: N^2
palmerasNieve sol: N  pred: N^2
-palmerasNieve2 sol: N  pred: N^2
--erasmus sol: N  pred: N^2
mergeSort sol: NLOGN  pred: N^2
nosInvaden sol: NLOGN  pred: N^2
11 / 15 => real error% of 73.33333333333333


In [ ]:
expected_train = []
for i in range(0,len(real_df)):
  high = real_df.tipoProblema.values[i]
  if high == 0:
    expected_train.append([1,0,0,0,0,0])
  elif high == 1:
    expected_train.append([0,1,0,0,0,0])
  elif high == 2:
    expected_train.append([0,0,1,0,0,0])
  elif high == 3:
    expected_train.append([0,0,0,1,0,0])
  elif high == 4:
    expected_train.append([0,0,0,0,1,0])
  elif high == 5:
    expected_train.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipoProblema.values[i]))


In [ ]:
#error teniendo en cuenta la aplicacion real del programa ( no todos los fallos son fallos )
#ejemplo: espero un lineal y entregan un n2, lo interpreto como n3, el resultado seria "no es lineal" y no seria un fallo
#creo que este tipo de error_count solo se puede calcular aplicando el modelo a un problema donde espero un resultado y entregan otro, no se como hacerlo aqui

#y_train = []            #tipo de ejecucion real del programa (realidad)
#expected_train = []     #tipo de ejecucion esperado por el problema (enunciado)
predictions_train = real_preds  #ordenes predecidos por la ia (prediccion)

correct = 0
partial_corrects = 0
errors = 0
for i in range(0, len(real_preds)):
  guess_id = np.argmax(predictions_train[i])
  answer_id = np.argmax(y_train[i])
  expected_id = np.argmax(expected_train[i])
  if guess_id == answer_id:
    correct += 1
  elif guess_id > answer_id and expected_id != answer_id:
    partial_corrects += 1
  else:
    errors += 1

print(str(errors) + " / " + str(len(real_preds)) + " => useful error% of " + str(errors/len(real_preds)*100))
print(str(correct) + " + " + str(partial_corrects) + " = " + str(correct+partial_corrects)  + " / " + str(len(real_preds)) + " => useful guesses% of " + str((correct+partial_corrects)/len(real_preds)*100))

#error_count = 0
#for i in range(0, len(real_preds)):
#  guess_id = np.argmax(real_preds[i])
#  answer_id = np.argmax(y_test[i])
#  if guess_id < answer_id:
#    error_count += 1
#print(str(error_count) + " / " + str(len(real_preds)) + " => useful error% of " + str(error_count/len(real_preds)*100))

1 / 15 => useful error% of 6.666666666666667
0 + 14 = 14 / 15 => useful guesses% of 93.33333333333333
